In [959]:
import pandas as pd 
import numpy as np
import datetime

from sklearn.metrics import roc_auc_score

In [960]:
# OrigencePocV2 final model scores on penfed applicants

final_model_path = '/d/shared/silver_projects_v2/origencepocv2/autoloanv1/modeling/model_artifacts/autoloan/model1Ensemble'
scores_df= pd.read_parquet(os.path.join(final_model_path, 'penfed_autoindirectv1_scores.parquet'))

In [961]:
ana = pd.read_parquet('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/processed/supporting_artifacts/analysis_data.parquet')
ana.shape

(651809, 179)

In [962]:
ana.flgFunded.value_counts()

flgFunded
0    497471
1    154338
Name: count, dtype: int64

In [963]:
# PenFed Sent Data
df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
df_raw.shape

/tmp/ipykernel_6138/4184390068.py:2: DtypeWarning: Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')


(651816, 126)

In [964]:
df_raw['PenFed_Funded'].value_counts()

PenFed_Funded
0    497478
1    154338
Name: count, dtype: int64

In [965]:
# Application Number not found in zest processed analysis data

df_raw['Application_Number'] = df_raw['Application_Number'].astype(str)
df_raw[~df_raw.Application_Number.isin(ana.appId)][['Application_Number',
                                                    'Application_Date','PenFed_Funded',
                                                    'Role_Code']].sort_values('Application_Number')

,Application_Number,Application_Date,PenFed_Funded,Role_Code
16967,110581107,2025-04-01,0,PRIMARY
580862,110581107,2025-04-01,0,JOINTOWN
15985,110581198,2025-04-01,0,PRIMARY
38399,110581199,2025-04-01,0,PRIMARY
25000,110581337,2025-04-01,0,PRIMARY
16206,110581576,2025-04-01,0,PRIMARY
90360,110581602,2025-04-01,0,PRIMARY


In [966]:
# Number of applications missing in zest analysis data 

df_raw[~df_raw.Application_Number.isin(ana.appId)].Application_Number.nunique()

6

# Filter Funded Target 

Joint Application: \
PenFed Funded Trade: Deduplicate to have one record.\
Proxy Trade: Find the proxy for primary and coborrower, then choose trade opened closest to app received date.\
Score/Attribute: Deduplicate by using the applicant with higher FICO/Vantage score. Use primary if they have the same score.\
Expected Result: One unique trade/score/attribute for joint application\
Total Count: 651,816 before joint application deduplication. \
Total Counts after joint application deduplication: 579,000.

In [967]:
print('Zest analysis data, before joint application deduplication:', ana.shape[0])
ana = ana.sort_values(['benchmark_scores', 'flgPrimary'], ascending=[False, False])
df = ana.drop_duplicates(['appId'])
print('Zest analysis data, Total Counts after joint application deduplication:', df.shape[0])

Zest analysis data, before joint application deduplication: 651809
Zest analysis data, Total Counts after joint application deduplication: 578994


First deduplicate for Joint applications with logic above.\
Exclude Application_Status  in ‘Refer’, ‘Processing’, ‘Exception’.\
Exclude FICO < 610 (Using higher fico for joint application
Python Code

**Majority of Refer is duplicate application**

df = df[df['Application_Status'].isin(['Refer', 'Processing', 'Exception'])==False]\
print(df.shape) # 561,137\
df = df[df['Max_Credit_Risk_Score']>=610] # 537656\
print(df.shape) # 537, 652


In [968]:
df['Application_Status'].value_counts()

Application_Status
Approve       342343
Decline       177408
Counter        41381
Refer          17567
Exception        294
Processing         1
Name: count, dtype: int64

In [969]:
df = df[df['Application_Status'].isin(['Refer', 'Processing', 'Exception'])==False]
print(df.shape) # 561,137

df['Max_Credit_Risk_Score'] = pd.to_numeric(df['Max_Credit_Risk_Score'], errors='coerce')
df = df[df['Max_Credit_Risk_Score']>=610] # 537656
print(df.shape) # 537, 652

(561132, 179)
(537647, 179)


**Additional Filters on PenFed Funded Loan (when PenFed_Funded=1)**\
- Exclude Insured Loan (exclude accounts with a lend pro number) 
- Exclude Canceled and Check Not Cashed 
- Exclude 3rd Party Fraud

PenFed Funded Only After filters in the filter section. 


In [970]:
for i in ['Lendpro_Cert_no', 'cancelled_flag', 'never_cash_flag', 'fraud_3rd_party']:
    print(df[i].value_counts(dropna = False))

Lendpro_Cert_no
nan          517258
NM0841978         2
RM1128448         2
AM1045431         2
CM0752118         2
              ...  
CM0819511         1
NM0807695         1
CM0770167         1
NM0814755         1
AM0929559         1
Name: count, Length: 20384, dtype: int64
cancelled_flag
nan    407464
0.0    130110
1.0        73
Name: count, dtype: int64
never_cash_flag
nan    407464
0.0    130090
1.0        93
Name: count, dtype: int64
fraud_3rd_party
nan    407464
0.0    130180
1.0         3
Name: count, dtype: int64


In [971]:
# Exclude some Funded loan
df = df[~((df['flgFunded']==1) & (df['Lendpro_Cert_no']!='nan'))] # exclude insured
print(df.flgFunded.value_counts())
df = df[~((df['flgFunded']==1) & (df['cancelled_flag']=='1.0'))] # exclude cancelled
df = df[~((df['flgFunded']==1) & (df['never_cash_flag']=='1.0'))] # exclude never cash the check
df = df[~((df['flgFunded']==1) & (df['fraud_3rd_party']=='1.0'))] # exclude 3rd party fraud
print(df.shape) # 517,119


flgFunded
0    407464
1    109794
Name: count, dtype: int64
(517114, 179)


In [972]:
df['Application_Fund_Year'] = df['Funded_Date'].str[:4]

pd.DataFrame(df[df['flgFunded'] == 1].\
             groupby(['Application_Fund_Year'])['appId'].count())

,appId
Application_Fund_Year,
2020,15
2021,37172
2022,41714
2023,21373
2024,8707
2025,669


# Create FICO and Partner Segment

By FICO Band: 610-649, 650-699, 700-739, 740-779, 780-850. Use and here’s the distribution based on full application.


In [973]:
fico_band = {
         "780-850":[780, 850],
         "740-779":[740, 779],
         "700-739":[700, 739],
         "650-699":[650, 699],
         "610-649":[610, 649]}

def assign_band(score):
    for band, (low, high) in fico_band.items():
        if low <= score <= high:
            return band
    return "Below 610"

In [974]:
df['fico_band'] = df['benchmark_scores'].apply(assign_band)
df['fico_band'].value_counts()

fico_band
780-850    150963
740-779    111541
650-699    111228
700-739    105997
610-649     37385
Name: count, dtype: int64

In [975]:
df['Seller_Name'].value_counts()

Seller_Name
Autopay Direct, Inc.                294246
Auto Approve                         70698
Caribou Financial Inc                46435
iLendingDIRECT                       34412
National Auto Loan Network           18119
Gravity Lending                      15303
RefiJet                              12934
Tresl                                 7719
Yield Solutions Group                 5993
Lending Arts                          5662
Innovative Funding Services, LLC      4914
Upfront                                466
FiWize                                 213
Name: count, dtype: int64

In [976]:
current_partners = {
    'Y': [
        'Autopay Direct, Inc.',
        'Auto Approve',
        'Caribou Financial Inc',
        'iLendingDIRECT',
        'Gravity Lending',
        'RefiJet'
    ],
    'N': [
        'National Auto Loan Network',
        'Tresl',
        'Yield Solutions Group',
        'Lending Arts',
        'Innovative Funding Services, LLC',
        'Upfront',
        'FiWize'
    ]
}
partner_map = {name: 'Y' for name in current_partners['Y']}
partner_map.update({name: 'N' for name in current_partners['N']})

In [977]:
df['current_partner'] = df['Seller_Name'].map(partner_map).fillna('Unknown')
df['current_partner'].value_counts()

current_partner
Y    474028
N     43086
Name: count, dtype: int64

# Reject Inference Only Segment 
- Further Separate by Approved Withdraw vs. Decline.

Please use the following table for category. 


Approved Withdraw\
PenFed Funded = 0 & [ 
    (Application_Status = Approve) or 
 (Application_Status = Counter and Adverse_Action_Reasons is None)]

234206

Decline\
PenFed Funded = 0 &
(Application_Status = Decline or 
 (Application_Status = Counter and Adverse_Action_Reasons Not None)

173263



In [978]:
# Validate using PenFed sent data

temp_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')
print(temp_raw.shape)

temp_raw = temp_raw.sort_values(['Credit_Risk_Score', 'Role_Code'], ascending=[False, True])
temp = temp_raw.drop_duplicates(['Application_Number'])

temp = temp[temp['Application_Status'].isin(['Refer', 'Processing', 'Exception'])==False]
print(temp.shape) # 561,137
temp = temp[temp['Max_Credit_Risk_Score']>=610] # 537656
print(temp.shape) # 537, 652

temp = temp[~((temp['PenFed_Funded']==1) & (temp['Lendpro_Cert_no'].notnull()))] # exclude insured
temp = temp[~((temp['PenFed_Funded']==1) & (temp['cancelled_flag']==1))] # exclude cancelled
temp = temp[~((temp['PenFed_Funded']==1) & (temp['never_cash_flag']==1))] # exclude never cash the check
temp = temp[~((temp['PenFed_Funded']==1) & (temp['fraud_3rd_party']==1))] # exclude 3rd party fraud
print(temp.shape) # 517,119


temp['Decision'] = None
temp['Decision'] = np.where(temp['PenFed_Funded']==1, 'Funded', temp['Decision'])
temp['Decision'] = np.where((temp['PenFed_Funded']==0) & 
                          ((temp['Application_Status']=='Approve')|((temp['Application_Status']=='Counter')&(temp['Adverse_Action_Reasons'].isnull()))), 'Withdraw', temp['Decision'])
temp['Decision'] = np.where((temp['PenFed_Funded']==0) & 
                          ((temp['Application_Status']=='Decline')|((temp['Application_Status']=='Counter')&(temp['Adverse_Action_Reasons'].notnull()))), 'Decline', temp['Decision'])

temp['Decision'].value_counts(dropna=False)

/tmp/ipykernel_6138/4027038549.py:3: DtypeWarning: Columns (21,42,60,61,71,72,122) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_raw = pd.read_csv('/d/shared/silver_projects_v2/penfed/autoindirectv1/shared_data/equifax/raw/client/data_3rd_party_refi_f_sent_20250523183131.csv')


(651816, 126)
(561137, 126)
(537652, 126)
(517119, 126)


Decision
Withdraw    234693
Decline     172776
Funded      109650
Name: count, dtype: int64

In [979]:
df['Decision'] = None
df['Decision'] = np.where(df['flgFunded']==1, 'Funded', df['Decision'])
df['Decision'] = np.where((df['flgFunded']==0) & 
                          ((df['Application_Status']=='Approve')|((df['Application_Status']=='Counter')&(df['Adverse_Action_Reasons']=='nan'))), 'Withdraw', df['Decision'])
df['Decision'] = np.where((df['flgFunded']==0) & 
                          ((df['Application_Status']=='Decline')|((df['Application_Status']=='Counter')&(df['Adverse_Action_Reasons']!='nan'))), 'Decline', df['Decision'])

df['Decision'].value_counts(dropna=False)


Decision
Withdraw    234689
Decline     172775
Funded      109650
Name: count, dtype: int64

# Funded Target Performance

## Target and Vintage

Time window: 
- Based on the Loan Booked date, not the application received date.
- Recent 1 year vs. Full time window.
- Separate by the segment above.


In [980]:
# Merge with Zest Pred 

df = df.merge(scores_df, left_index = True, right_index = True, how = 'left')
df.shape

(517114, 186)

In [981]:
df['inverted_benchmark_scores'] = - df['benchmark_scores']
df['inverted_benchmark_scores'].describe()

count    517114.000000
mean       -738.536987
std          59.284409
min        -850.000000
25%        -787.000000
50%        -741.000000
75%        -691.000000
max        -610.000000
Name: inverted_benchmark_scores, dtype: float64

In [982]:
df_book = df[df.flgFunded == 1.0]
df_book.shape

(109650, 187)

In [983]:
df_book['Funded_Year'] =  df['Funded_Date'].str[:4]
df_book['Funded_Month'] =  df['Funded_Date'].str[:7]

df_book['Funded_Date'] = pd.to_datetime(df_book['Funded_Date'], format='%Y-%m-%d')

/tmp/ipykernel_6138/2144837808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_book['Funded_Year'] =  df['Funded_Date'].str[:4]
/tmp/ipykernel_6138/2144837808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_book['Funded_Month'] =  df['Funded_Date'].str[:7]
/tmp/ipykernel_6138/2144837808.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [984]:
target_cols = ['mob_6_60', 'mob_12_90', 'mob_18_90', 'mob_24_120', 'mob_12_120', 'mob_18_120']
for i in target_cols: 
    df_book[i] = pd.to_numeric(df_book[i], errors='coerce')

/tmp/ipykernel_6138/2468960345.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_book[i] = pd.to_numeric(df_book[i], errors='coerce')


In [985]:
df_book[(df_book['Funded_Date']>=datetime.datetime(2023,10,1)) & 
        (df_book['Funded_Date']<=datetime.datetime(2024,9,30))].groupby(['Funded_Year', 'Funded_Month'])['mob_6_60'].agg(['count', 'sum', 'mean'])


count  sum      mean
Funded_Year Funded_Month                      
2023        2023-10        1431  3.0  0.002096
            2023-11        1027  3.0  0.002921
            2023-12         976  3.0  0.003074
2024        2024-01        1241  7.0  0.005641
            2024-02        1372  6.0  0.004373
            2024-03        1517  2.0  0.001318
            2024-04        1426  4.0  0.002805
            2024-05         922  2.0  0.002169
            2024-06         479  0.0  0.000000
            2024-07         472  0.0  0.000000
            2024-08         435  0.0  0.000000
            2024-09         293  0.0  0.000000

In [986]:
def get_roc_auc( y_true, y_score):
    try:
        if len(set(y_true)) < 2:
            return np.nan
        return roc_auc_score(y_true, y_score)
    except:
        return None

def compute_cols(group, target_col):
    return pd.Series({
        'benchmark_auc': get_roc_auc(group[target_col], group['inverted_benchmark_scores']),
        'zest_model_auc': get_roc_auc(group[target_col], group['final_model_predictions']),
        'app_count': group[target_col].count(),
        'bad_count': group[target_col].sum(),
        'target_rate': group[target_col].mean(),

    })

def performance_tables(df, fund_col, start_time, end_time, target_column, decision_seg, fico_seg, seller_seg, 
                      partner_seg): 
    print(target_column)
    
    seg_df = df[(df[fund_col]>=start_time) & (df[fund_col]<=end_time)]
    
    start_date_str = start_time.strftime('%Y-%m-%d')
    end_date_str = end_time.strftime('%Y-%m-%d')
    
   
    target_summary = pd.DataFrame({
        'app_with_proxy_auto_loan_count': seg_df[target_column].count(),
        f'{target_column}_bad_count': seg_df[target_column].sum(),
        f'{target_column}_target_rate': seg_df[target_column].mean(),}, index = [f'{start_date_str} to {end_date_str}'])
    
    if seg_df[target_column].isnull().any(): 
        seg_df = seg_df[~seg_df[target_column].isnull()]
    
    overall_auc_df = pd.DataFrame([compute_cols(seg_df, target_column)], index = [f'{start_date_str} to {end_date_str}'])
    
    results_by_decision = pd.DataFrame()
    if decision_seg == True: 
        results_by_decision = seg_df.groupby('Decision').apply(lambda group: compute_cols(group, target_column))
    
    results_by_fico_band = pd.DataFrame()
    if fico_seg == True: 
        results_by_fico_band = seg_df.groupby('fico_band').apply(lambda group: compute_cols(group, target_column))
    
    results_by_seller = pd.DataFrame()
    if seller_seg == True:
        results_by_seller = seg_df.groupby('Seller_Name').apply(lambda group: compute_cols(group, target_column))
    
    results_by_current_partner = pd.DataFrame()
    if partner_seg == True: 
        results_by_current_partner = seg_df.groupby('current_partner').apply(lambda group: compute_cols(group, target_column))
 
    final_df = pd.concat(
        { 
            'Overall': overall_auc_df,
            'Decision': results_by_decision,
            'fico_band': results_by_fico_band,
            'Seller_Name': results_by_seller,
            'current_partner': results_by_current_partner,
        },
        names=['grouping', 'group'])
    
    return final_df

## 1. Most Recent 12 Month: Oct 2023 - Sept 2024

In [987]:
# Most Recent 12 Month: Oct 2023 - Sept 2024

start_time = datetime.datetime(2023,10,1)
end_time = datetime.datetime(2024,9,30)

target_column = 'mob_6_60'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_6_60


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-10-01 to 2024-09-30         0.689254        0.748468   
Decision        Funded                           0.689254        0.748468   
fico_band       650-699                               NaN             NaN   
                700-739                          0.431818        0.090909   
                740-779                          0.594710        0.679310   
                780-850                          0.617086        0.782598   
Seller_Name     Auto Approve                     0.700632        0.675783   
                Autopay Direct, Inc.             0.718085        0.856865   
                Caribou Financial Inc            0.754491        0.411843   
                Gravity Lending                  0.953072        0.976109   
                National Auto Loan Network            NaN             NaN   
                RefiJet                          0.064611        0.641703   
                iLendingDIRECT                   0.532828        0.602904   
current_partner N                                     NaN             NaN   
                Y                                0.690740        0.750751   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-10-01 to 2024-09-30      11591.0       30.0     0.002588  
Decision        Funded                        11591.0       30.0     0.002588  
fico_band       650-699                           9.0        0.0     0.000000  
                700-739                         112.0        2.0     0.017857  
                740-779                        4119.0       17.0     0.004127  
                780-850                        7351.0       11.0     0.001496  
Seller_Name     Auto Approve                   2903.0       11.0     0.003789  
                Autopay Direct, Inc.           5494.0       12.0     0.002184  
                Caribou Financial Inc          1005.0        3.0     0.002985  
                Gravity Lending                 587.0        1.0     0.001704  
                National Auto Loan Network      126.0        0.0     0.000000  
                RefiJet                         682.0        1.0     0.001466  
                iLendingDIRECT                  794.0        2.0     0.002519  
current_partner N                               126.0        0.0     0.000000  
                Y                             11465.0       30.0     0.002617

## 2. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2022 – Mar 2024


In [988]:
# Most Recent 12 Month: April 2023 - Mar 2024

start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_90


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.707803        0.759528   
Decision        Funded                           0.707803        0.759528   
fico_band       610-649                               NaN             NaN   
                650-699                          0.555556        0.944444   
                700-739                          0.546524        0.649160   
                740-779                          0.546455        0.679791   
                780-850                          0.658405        0.769259   
Seller_Name     Auto Approve                     0.665882        0.713324   
                Autopay Direct, Inc.             0.734052        0.800382   
                Caribou Financial Inc            0.641150        0.730246   
                Gravity Lending                  0.747863        0.777498   
                Lending Arts                     0.666667        0.500000   
                National Auto Loan Network       0.627181        0.750842   
                RefiJet                          0.688480        0.673774   
                Tresl                            0.875000        1.000000   
                iLendingDIRECT                   0.716625        0.765337   
current_partner N                                0.642857        0.726010   
                Y                                0.707043        0.757515   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      17199.0      262.0     0.015233  
Decision        Funded                        17199.0      262.0     0.015233  
fico_band       610-649                           1.0        0.0     0.000000  
                650-699                          19.0        1.0     0.052632  
                700-739                        2836.0      103.0     0.036319  
                740-779                        7179.0      122.0     0.016994  
                780-850                        7164.0       36.0     0.005025  
Seller_Name     Auto Approve                   4283.0       75.0     0.017511  
                Autopay Direct, Inc.           7021.0       78.0     0.011110  
                Caribou Financial Inc          2133.0       37.0     0.017346  
                Gravity Lending                 802.0        9.0     0.011222  
                Lending Arts                     26.0        2.0     0.076923  
                National Auto Loan Network      308.0       11.0     0.035714  
                RefiJet                        1173.0       17.0     0.014493  
                Tresl                             9.0        1.0     0.111111  
                iLendingDIRECT                 1444.0       32.0     0.022161  
current_partner N                               343.0       14.0     0.040816  
                Y                             16856.0      248.0     0.014713

In [989]:
# Full Time Window: Jan 2022 – Mar 2024


start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_90


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.654070   
Decision        Funded                                 0.654070   
fico_band       610-649                                0.515723   
                650-699                                0.550339   
                700-739                                0.525217   
                740-779                                0.523982   
                780-850                                0.659405   
Seller_Name     Auto Approve                           0.623229   
                Autopay Direct, Inc.                   0.668799   
                Caribou Financial Inc                  0.632223   
                FiWize                                      NaN   
                Gravity Lending                        0.708506   
                Innovative Funding Services, LLC       0.965909   
                Lending Arts                           0.564799   
                National Auto Loan Network             0.595892   
                RefiJet                                0.653846   
                Tresl                                  0.587083   
                Upfront                                0.695896   
                Yield Solutions Group                  0.628101   
                iLendingDIRECT                         0.665775   
current_partner N                                      0.616183   
                Y                                      0.656915   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.730658    65753.0   
Decision        Funded                                  0.730658    65753.0   
fico_band       610-649                                 0.515723       56.0   
                650-699                                 0.672301    14912.0   
                700-739                                 0.670372    20211.0   
                740-779                                 0.704405    18145.0   
                780-850                                 0.745292    12429.0   
Seller_Name     Auto Approve                            0.670457    11161.0   
                Autopay Direct, Inc.                    0.751531    24641.0   
                Caribou Financial Inc                   0.734599    10909.0   
                FiWize                                       NaN        5.0   
                Gravity Lending                         0.701535     2503.0   
                Innovative Funding Services, LLC        0.750000       45.0   
                Lending Arts                            0.667737     1121.0   
                National Auto Loan Network              0.674658     2314.0   
                RefiJet                                 0.681844     1963.0   
                Tresl                                   0.783463     2140.0   
                Upfront                                 0.727612      138.0   
                Yield Solutions Group                   0.690798     1143.0   
                iLendingDIRECT                          0.752625     7670.0   
current_partner N                                       0.715491     6906.0   
                Y                                       0.730989    58847.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             1556.0     0.023664  
Decision        Funded                               1556.0     0.023664  
fico_band       610-649                                 3.0     0.053571  
                650-699                               604.0     0.040504  
                700-739                               570.0     0.028202  
                740

## 3. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2022 – Mar 2024


In [990]:
# Most Recent 12 Month: April 2023 - Mar 2024


start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.704373        0.754549   
Decision        Funded                           0.704373        0.754549   
fico_band       610-649                               NaN             NaN   
                650-699                          0.555556        0.944444   
                700-739                          0.555043        0.640011   
                740-779                          0.546646        0.672404   
                780-850                          0.658405        0.769259   
Seller_Name     Auto Approve                     0.667390        0.708135   
                Autopay Direct, Inc.             0.726876        0.796569   
                Caribou Financial Inc            0.633331        0.732861   
                Gravity Lending                  0.723473        0.749055   
                Lending Arts                     0.666667        0.500000   
                National Auto Loan Network       0.655705        0.750000   
                RefiJet                          0.688480        0.673774   
                Tresl                            0.875000        1.000000   
                iLendingDIRECT                   0.707691        0.758715   
current_partner N                                0.665967        0.723077   
                Y                                0.702682        0.752638   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      17199.0      250.0     0.014536  
Decision        Funded                        17199.0      250.0     0.014536  
fico_band       610-649                           1.0        0.0     0.000000  
                650-699                          19.0        1.0     0.052632  
                700-739                        2836.0       97.0     0.034203  
                740-779                        7179.0      116.0     0.016158  
                780-850                        7164.0       36.0     0.005025  
Seller_Name     Auto Approve                   4283.0       73.0     0.017044  
                Autopay Direct, Inc.           7021.0       73.0     0.010397  
                Caribou Financial Inc          2133.0       35.0     0.016409  
                Gravity Lending                 802.0        8.0     0.009975  
                Lending Arts                     26.0        2.0     0.076923  
                National Auto Loan Network      308.0       10.0     0.032468  
                RefiJet                        1173.0       17.0     0.014493  
                Tresl                             9.0        1.0     0.111111  
                iLendingDIRECT                 1444.0       31.0     0.021468  
current_partner N                               343.0       13.0     0.037901  
                Y                             16856.0      237.0     0.014060

In [991]:
# Full Time Window: Jan 2022 – Mar 2024


start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.638121   
Decision        Funded                                 0.638121   
fico_band       610-649                                0.515723   
                650-699                                0.562540   
                700-739                                0.523076   
                740-779                                0.516945   
                780-850                                0.666683   
Seller_Name     Auto Approve                           0.615086   
                Autopay Direct, Inc.                   0.651789   
                Caribou Financial Inc                  0.610004   
                FiWize                                      NaN   
                Gravity Lending                        0.692883   
                Innovative Funding Services, LLC            NaN   
                Lending Arts                           0.568658   
                National Auto Loan Network             0.595271   
                RefiJet                                0.662071   
                Tresl                                  0.596957   
                Upfront                                0.656250   
                Yield Solutions Group                  0.606203   
                iLendingDIRECT                         0.647938   
current_partner N                                      0.610218   
                Y                                      0.639526   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.722117    65753.0   
Decision        Funded                                  0.722117    65753.0   
fico_band       610-649                                 0.515723       56.0   
                650-699                                 0.671160    14912.0   
                700-739                                 0.667962    20211.0   
                740-779                                 0.699184    18145.0   
                780-850                                 0.749418    12429.0   
Seller_Name     Auto Approve                            0.666759    11161.0   
                Autopay Direct, Inc.                    0.749742    24641.0   
                Caribou Financial Inc                   0.723150    10909.0   
                FiWize                                       NaN        5.0   
                Gravity Lending                         0.679853     2503.0   
                Innovative Funding Services, LLC             NaN       45.0   
                Lending Arts                            0.647403     1121.0   
                National Auto Loan Network              0.665212     2314.0   
                RefiJet                                 0.680160     1963.0   
                Tresl                                   0.746157     2140.0   
                Upfront                                 0.886029      138.0   
                Yield Solutions Group                   0.669907     1143.0   
                iLendingDIRECT                          0.741255     7670.0   
current_partner N                                       0.701102     6906.0   
                Y                                       0.723150    58847.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             1273.0     0.019360  
Decision        Funded                               1273.0     0.019360  
fico_band       610-649                                 3.0     0.053571  
                650-699                               452.0     0.030311  
                700-739                               484.0     0.023947  
                740

## 4. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023


In [992]:
# Most Recent 12 Month: Oct 2022 - Sept 2023

target_column = 'mob_18_90'

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_90


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.638698        0.700792   
Decision        Funded                           0.638698        0.700792   
fico_band       610-649                               NaN             NaN   
                650-699                          0.635484        0.690323   
                700-739                          0.534688        0.646588   
                740-779                          0.524345        0.659017   
                780-850                          0.652269        0.715050   
Seller_Name     Auto Approve                     0.614826        0.667864   
                Autopay Direct, Inc.             0.646987        0.725854   
                Caribou Financial Inc            0.618860        0.681412   
                Gravity Lending                  0.652884        0.664396   
                Lending Arts                     0.641250        0.704167   
                National Auto Loan Network       0.554194        0.616173   
                RefiJet                          0.640694        0.669918   
                Tresl                            0.666667        0.952381   
                iLendingDIRECT                   0.640142        0.722835   
current_partner N                                0.575591        0.653973   
                Y                                0.640597        0.701757   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      21149.0      860.0     0.040664  
Decision        Funded                        21149.0      860.0     0.040664  
fico_band       610-649                           1.0        0.0     0.000000  
                650-699                          36.0        5.0     0.138889  
                700-739                        8180.0      493.0     0.060269  
                740-779                        7916.0      289.0     0.036508  
                780-850                        5016.0       73.0     0.014553  
Seller_Name     Auto Approve                   5253.0      245.0     0.046640  
                Autopay Direct, Inc.           7631.0      250.0     0.032761  
                Caribou Financial Inc          2936.0      106.0     0.036104  
                Gravity Lending                 812.0       25.0     0.030788  
                Lending Arts                    206.0        6.0     0.029126  
                National Auto Loan Network      518.0       38.0     0.073359  
                RefiJet                        1291.0       53.0     0.041053  
                Tresl                            23.0        2.0     0.086957  
                iLendingDIRECT                 2479.0      135.0     0.054457  
current_partner N                               747.0       46.0     0.061580  
                Y                             20402.0      814.0     0.039898

In [993]:
#  Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'mob_18_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_90


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.632705   
Decision        Funded                                 0.632705   
fico_band       610-649                                0.492157   
                650-699                                0.532771   
                700-739                                0.531663   
                740-779                                0.514261   
                780-850                                0.645101   
Seller_Name     Auto Approve                           0.609871   
                Autopay Direct, Inc.                   0.635500   
                Caribou Financial Inc                  0.636903   
                FiWize                                      NaN   
                Gravity Lending                        0.636759   
                Innovative Funding Services, LLC       0.965909   
                Lending Arts                           0.556144   
                National Auto Loan Network             0.583585   
                RefiJet                                0.639624   
                Tresl                                  0.621850   
                Upfront                                0.653217   
                Yield Solutions Group                  0.628372   
                iLendingDIRECT                         0.632307   
current_partner N                                      0.616127   
                Y                                      0.633868   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.718119    58406.0   
Decision        Funded                                  0.718119    58406.0   
fico_band       610-649                                 0.372549       56.0   
                650-699                                 0.670264    14905.0   
                700-739                                 0.678665    20143.0   
                740-779                                 0.697281    14874.0   
                780-850                                 0.712651     8428.0   
Seller_Name     Auto Approve                            0.672917     9204.0   
                Autopay Direct, Inc.                    0.729269    21677.0   
                Caribou Financial Inc                   0.727767    10064.0   
                FiWize                                       NaN        5.0   
                Gravity Lending                         0.671551     2117.0   
                Innovative Funding Services, LLC        0.750000       45.0   
                Lending Arts                            0.706361     1121.0   
                National Auto Loan Network              0.662202     2211.0   
                RefiJet                                 0.650102     1448.0   
                Tresl                                   0.773159     2140.0   
                Upfront                                 0.818975      138.0   
                Yield Solutions Group                   0.692602     1143.0   
                iLendingDIRECT                          0.731346     7093.0   
current_partner N                                       0.719105     6803.0   
                Y                                       0.716826    51603.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             2786.0     0.047701  
Decision        Funded                               2786.0     0.047701  
fico_band       610-649                                 5.0     0.089286  
                650-699                              1107.0     0.074270  
                700-739                              1066.0     0.052922  
                740

## 5. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023



In [994]:
# Most Recent 12 Month: Oct 2022 - Sept 2023
target_column = 'mob_18_120'

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.638188        0.698832   
Decision        Funded                           0.638188        0.698832   
fico_band       610-649                               NaN             NaN   
                650-699                          0.635484        0.690323   
                700-739                          0.534064        0.641961   
                740-779                          0.528779        0.658677   
                780-850                          0.659502        0.715342   
Seller_Name     Auto Approve                     0.610876        0.669264   
                Autopay Direct, Inc.             0.650891        0.727319   
                Caribou Financial Inc            0.617817        0.681246   
                Gravity Lending                  0.645463        0.650169   
                Lending Arts                     0.712438        0.690547   
                National Auto Loan Network       0.570423        0.625732   
                RefiJet                          0.636397        0.655487   
                Tresl                            0.666667        0.952381   
                iLendingDIRECT                   0.633283        0.713414   
current_partner N                                0.598024        0.659439   
                Y                                0.639022        0.699667   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      21149.0      811.0     0.038347  
Decision        Funded                        21149.0      811.0     0.038347  
fico_band       610-649                           1.0        0.0     0.000000  
                650-699                          36.0        5.0     0.138889  
                700-739                        8180.0      463.0     0.056601  
                740-779                        7916.0      274.0     0.034613  
                780-850                        5016.0       69.0     0.013756  
Seller_Name     Auto Approve                   5253.0      232.0     0.044165  
                Autopay Direct, Inc.           7631.0      236.0     0.030926  
                Caribou Financial Inc          2936.0      104.0     0.035422  
                Gravity Lending                 812.0       24.0     0.029557  
                Lending Arts                    206.0        5.0     0.024272  
                National Auto Loan Network      518.0       35.0     0.067568  
                RefiJet                        1291.0       48.0     0.037180  
                Tresl                            23.0        2.0     0.086957  
                iLendingDIRECT                 2479.0      125.0     0.050424  
current_partner N                               747.0       42.0     0.056225  
                Y                             20402.0      769.0     0.037692

In [995]:
#  Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'mob_18_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.619596   
Decision        Funded                                 0.619596   
fico_band       610-649                                0.492157   
                650-699                                0.533801   
                700-739                                0.526956   
                740-779                                0.519466   
                780-850                                0.650766   
Seller_Name     Auto Approve                           0.597620   
                Autopay Direct, Inc.                   0.622672   
                Caribou Financial Inc                  0.622837   
                FiWize                                      NaN   
                Gravity Lending                        0.618468   
                Innovative Funding Services, LLC       0.965909   
                Lending Arts                           0.548925   
                National Auto Loan Network             0.575510   
                RefiJet                                0.636800   
                Tresl                                  0.618715   
                Upfront                                0.592803   
                Yield Solutions Group                  0.642558   
                iLendingDIRECT                         0.621107   
current_partner N                                      0.611261   
                Y                                      0.620027   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.712310    58406.0   
Decision        Funded                                  0.712310    58406.0   
fico_band       610-649                                 0.372549       56.0   
                650-699                                 0.674338    14905.0   
                700-739                                 0.676728    20143.0   
                740-779                                 0.692187    14874.0   
                780-850                                 0.715292     8428.0   
Seller_Name     Auto Approve                            0.673290     9204.0   
                Autopay Direct, Inc.                    0.725576    21677.0   
                Caribou Financial Inc                   0.720495    10064.0   
                FiWize                                       NaN        5.0   
                Gravity Lending                         0.648909     2117.0   
                Innovative Funding Services, LLC        0.750000       45.0   
                Lending Arts                            0.692854     1121.0   
                National Auto Loan Network              0.664783     2211.0   
                RefiJet                                 0.638226     1448.0   
                Tresl                                   0.759480     2140.0   
                Upfront                                 0.819444      138.0   
                Yield Solutions Group                   0.680016     1143.0   
                iLendingDIRECT                          0.728114     7093.0   
current_partner N                                       0.711432     6803.0   
                Y                                       0.711488    51603.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             2412.0     0.041297  
Decision        Funded                               2412.0     0.041297  
fico_band       610-649                                 5.0     0.089286  
                650-699                               902.0     0.060517  
                700-739                               947.0     0.047014  
                740

## 6. Most Recent 12 Month: April 2022 - Mar 2023; Full Time Window: Jan 2022 – Mar 2023

In [996]:
# Most Recent 12 Month: April 2022 - Mar 2023

start_time = datetime.datetime(2022,4,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'mob_24_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_24_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-04-01 to 2023-03-01         0.605697        0.695282   
Decision        Funded                           0.605697        0.695282   
fico_band       610-649                          0.460000        0.440000   
                650-699                          0.537939        0.660061   
                700-739                          0.536939        0.674223   
                740-779                          0.517066        0.688227   
                780-850                          0.566633        0.682178   
Seller_Name     Auto Approve                     0.607882        0.679142   
                Autopay Direct, Inc.             0.602496        0.697115   
                Caribou Financial Inc            0.617466        0.706673   
                Gravity Lending                  0.557831        0.607916   
                Lending Arts                     0.525775        0.717032   
                National Auto Loan Network       0.562615        0.667613   
                RefiJet                          0.580217        0.639261   
                Tresl                            0.597386        0.712918   
                Upfront                          0.770093        0.893458   
                Yield Solutions Group            0.601128        0.666377   
                iLendingDIRECT                   0.608989        0.701691   
current_partner N                                0.594160        0.702688   
                Y                                0.606074        0.693017   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-04-01 to 2023-03-01      29336.0     2129.0     0.072573  
Decision        Funded                        29336.0     2129.0     0.072573  
fico_band       610-649                          28.0        3.0     0.107143  
                650-699                        8011.0      831.0     0.103732  
                700-739                       11447.0      848.0     0.074081  
                740-779                        6979.0      360.0     0.051583  
                780-850                        2871.0       87.0     0.030303  
Seller_Name     Auto Approve                   4696.0      335.0     0.071337  
                Autopay Direct, Inc.          10952.0      771.0     0.070398  
                Caribou Financial Inc          4689.0      300.0     0.063980  
                Gravity Lending                 861.0       51.0     0.059233  
                Lending Arts                    621.0       29.0     0.046699  
                National Auto Loan Network     1278.0      155.0     0.121283  
                RefiJet                         628.0       45.0     0.071656  
                Tresl                           927.0       60.0     0.064725  
                Upfront                         112.0        5.0     0.044643  
                Yield Solutions Group           619.0       55.0     0.088853  
                iLendingDIRECT                 3953.0      323.0     0.081710  
current_partner N                              3557.0      304.0     0.085465  
                Y                             25779.0     1825.0     0.070794

In [997]:
# Full Time Window: Jan 2022 – Mar 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'mob_24_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_book, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_24_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-03-01               0.614911   
Decision        Funded                                 0.614911   
fico_band       610-649                                0.565476   
                650-699                                0.538931   
                700-739                                0.537975   
                740-779                                0.513025   
                780-850                                0.594405   
Seller_Name     Auto Approve                           0.606064   
                Autopay Direct, Inc.                   0.604494   
                Caribou Financial Inc                  0.618011   
                FiWize                                      NaN   
                Gravity Lending                        0.618011   
                Innovative Funding Services, LLC       0.965909   
                Lending Arts                           0.558618   
                National Auto Loan Network             0.570816   
                RefiJet                                0.580217   
                Tresl                                  0.615486   
                Upfront                                0.660096   
                Yield Solutions Group                  0.644808   
                iLendingDIRECT                         0.618699   
current_partner N                                      0.617750   
                Y                                      0.613969   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-03-01                0.706846    45817.0   
Decision        Funded                                  0.706846    45817.0   
fico_band       610-649                                 0.476190       55.0   
                650-699                                 0.672529    14886.0   
                700-739                                 0.675368    16185.0   
                740-779                                 0.704141     9944.0   
                780-850                                 0.696202     4747.0   
Seller_Name     Auto Approve                            0.683583     6168.0   
                Autopay Direct, Inc.                    0.706652    16576.0   
                Caribou Financial Inc                   0.709141     8438.0   
                FiWize                                       NaN        5.0   
                Gravity Lending                         0.676065     1585.0   
                Innovative Funding Services, LLC        0.750000       45.0   
                Lending Arts                            0.716329     1061.0   
                National Auto Loan Network              0.659606     1953.0   
                RefiJet                                 0.639261      628.0   
                Tresl                                   0.739790     2128.0   
                Upfront                                 0.818269      138.0   
                Yield Solutions Group                   0.698575     1143.0   
                iLendingDIRECT                          0.711896     5949.0   
current_partner N                                       0.715977     6473.0   
                Y                                       0.704723    39344.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-03-01             3073.0     0.067071  
Decision        Funded                               3073.0     0.067071  
fico_band       610-649                                 7.0     0.127273  
                650-699                              1422.0     0.095526  
                700-739                              1075.0     0.066420  
                740

# Unfunded Target Performance

Reject Inference: Only use the Auto Loan Proxy. For duplication on Auto Loan Proxy, choose trade opened closest to app received date 

In [998]:
df_unbook = df[df.flgFunded == 0.0]
df_unbook.shape

(407464, 187)

In [999]:
proxy_loans_df = pd.read_parquet('/d/shared/users/lyt/penfed_poc1_unfunded_proxy_targets/auto_proxy_targets.parquet')
req_cols = ['ZEST_KEY', 'DATE_OPENED',
            'proxy_DQ60_m6', 'proxy_DQ60_m6_status',
            'proxy_DQ90_m12', 'proxy_DQ90_m12_status',
            'proxy_DQ120_m12', 'proxy_DQ120_m12_status',
            'proxy_DQ90_m18', 'proxy_DQ90_m18_status',
            'proxy_DQ120_m18', 'proxy_DQ120_m18_status',
            'proxy_DQ120_m24', 'proxy_DQ120_m24_status',
            'PAYMENT_HISTORY_1_24', 'PAYMENT_HISTORY_25_36', 'PAYMENT_HISTORY_37_48']
proxy_loans_df = proxy_loans_df[req_cols]

In [1000]:
print('Auto Proxy Match Rate')
df_unbook.loc[:, 'auto_proxy_found'] = np.where(df_unbook.index.isin(proxy_loans_df.ZEST_KEY), 
                                              'yes',
                                              'no')

df_unbook[ 'auto_proxy_found'].value_counts(normalize = True)

Auto Proxy Match Rate


/tmp/ipykernel_6138/2206635650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unbook.loc[:, 'auto_proxy_found'] = np.where(df_unbook.index.isin(proxy_loans_df.ZEST_KEY),


auto_proxy_found
no     0.711852
yes    0.288148
Name: proportion, dtype: float64

In [1001]:
df_unbook_proxy = df_unbook.merge(proxy_loans_df, left_index = True, 
                                  right_on = 'ZEST_KEY',
                                  how = 'left').reset_index(drop = True)

In [1002]:
df_unbook_proxy.ZEST_KEY.nunique()

407464

In [1003]:
df_unbook_proxy.shape

(424334, 205)

In [1004]:
df_unbook_proxy['abs_appdate_minus_opendate'] = (df_unbook_proxy['DATE_OPENED'] - df_unbook_proxy['appDate']).abs().dt.days
df_unbook_proxy['abs_appdate_minus_opendate'].describe()

count    134280.000000
mean         22.531941
std          23.613301
min           0.000000
25%           5.000000
50%          14.000000
75%          31.000000
max          90.000000
Name: abs_appdate_minus_opendate, dtype: float64

In [1005]:
# Sort based on the minimum date difference between application date and trade open date
# Then keep the tradelines with worst performance

df_unbook_w_targets = df_unbook_proxy.sort_values(['ZEST_KEY', 'abs_appdate_minus_opendate', 'proxy_DQ60_m6'],
                                   ascending = [True, True, False]).drop_duplicates(['ZEST_KEY'],keep='first')


In [1006]:
df_unbook_w_targets.shape, df_unbook_w_targets.ZEST_KEY.nunique()

((407464, 206), 407464)

In [1007]:
df_unbook_w_targets['Decision'].value_counts()

Decision
Withdraw    234689
Decline     172775
Name: count, dtype: int64

## 1. Most Recent 12 Month: Oct 2023 - Sept 2024

In [1008]:
# Most Recent 12 Month: Oct 2023 - Sept 2024

start_time = datetime.datetime(2023,10,1)
end_time = datetime.datetime(2024,9,30)

target_column = 'proxy_DQ60_m6'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ60_m6


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-10-01 to 2024-09-30         0.702710        0.780864   
Decision        Decline                          0.596889        0.712459   
                Withdraw                         0.734585        0.781102   
fico_band       610-649                          0.459936        0.477564   
                650-699                          0.422542        0.713485   
                700-739                          0.618603        0.721366   
                740-779                          0.625589        0.726411   
                780-850                          0.709374        0.770909   
Seller_Name     Auto Approve                     0.652625        0.697402   
                Autopay Direct, Inc.             0.704205        0.801531   
                Caribou Financial Inc            0.411352        0.363520   
                Gravity Lending                  0.820265        0.725395   
                National Auto Loan Network       0.596836        0.641844   
                RefiJet                          0.703218        0.972649   
                Tresl                                 NaN             NaN   
                iLendingDIRECT                        NaN             NaN   
current_partner N                                0.597495        0.642429   
                Y                                0.694896        0.777626   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-10-01 to 2024-09-30      23906.0       76.0     0.003179  
Decision        Decline                       10132.0       58.0     0.005724  
                Withdraw                      13774.0       18.0     0.001307  
fico_band       610-649                         107.0        3.0     0.028037  
                650-699                         934.0       12.0     0.012848  
                700-739                        2774.0       13.0     0.004686  
                740-779                        6407.0       22.0     0.003434  
                780-850                       13684.0       26.0     0.001900  
Seller_Name     Auto Approve                   3909.0       19.0     0.004861  
                Autopay Direct, Inc.          14980.0       40.0     0.002670  
                Caribou Financial Inc           592.0        4.0     0.006757  
                Gravity Lending                1036.0        3.0     0.002896  
                National Auto Loan Network      617.0        6.0     0.009724  
                RefiJet                        2024.0        4.0     0.001976  
                Tresl                             1.0        0.0     0.000000  
                iLendingDIRECT                  747.0        0.0     0.000000  
current_partner N                               618.0        6.0     0.009709  
                Y                             23288.0       70.0     0.003006

## 2. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2022 – Mar 2024




In [1009]:
# Most Recent 12 Month: April 2023 - Mar 2024

start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'proxy_DQ90_m12'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ90_m12


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.713163        0.802773   
Decision        Decline                          0.599670        0.731769   
                Withdraw                         0.750507        0.809893   
fico_band       610-649                          0.567643        0.730415   
                650-699                          0.553661        0.660345   
                700-739                          0.548292        0.726666   
                740-779                          0.599133        0.692891   
                780-850                          0.654258        0.898568   
Seller_Name     Auto Approve                     0.624323        0.775659   
                Autopay Direct, Inc.             0.763960        0.827703   
                Caribou Financial Inc            0.487678        0.617195   
                Gravity Lending                  0.577470        0.673452   
                Lending Arts                     0.767241        0.758621   
                National Auto Loan Network       0.517945        0.642314   
                RefiJet                          0.801114        0.799095   
                Tresl                                 NaN             NaN   
                iLendingDIRECT                   0.693526        0.808468   
current_partner N                                0.530720        0.652086   
                Y                                0.709437        0.802714   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      25361.0      288.0     0.011356  
Decision        Decline                        9709.0      195.0     0.020084  
                Withdraw                      15652.0       93.0     0.005942  
fico_band       610-649                         231.0       14.0     0.060606  
                650-699                        1761.0       49.0     0.027825  
                700-739                        4891.0       92.0     0.018810  
                740-779                        7525.0       90.0     0.011960  
                780-850                       10953.0       43.0     0.003926  
Seller_Name     Auto Approve                   4046.0       61.0     0.015077  
                Autopay Direct, Inc.          13922.0       98.0     0.007039  
                Caribou Financial Inc           895.0       25.0     0.027933  
                Gravity Lending                1277.0       12.0     0.009397  
                Lending Arts                     59.0        1.0     0.016949  
                National Auto Loan Network     1376.0       52.0     0.037791  
                RefiJet                        2612.0       19.0     0.007274  
                Tresl                            38.0        0.0     0.000000  
                iLendingDIRECT                 1136.0       20.0     0.017606  
current_partner N                              1473.0       53.0     0.035981  
                Y                             23888.0      235.0     0.009838

In [1010]:
# Full Time Window: Jan 2022 – Mar 2024

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'proxy_DQ90_m12'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ90_m12


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.714303   
Decision        Decline                                0.630217   
                Withdraw                               0.747112   
fico_band       610-649                                0.541537   
                650-699                                0.536851   
                700-739                                0.556339   
                740-779                                0.574576   
                780-850                                0.654356   
Seller_Name     Auto Approve                           0.641416   
                Autopay Direct, Inc.                   0.761087   
                Caribou Financial Inc                  0.631163   
                FiWize                                      NaN   
                Gravity Lending                        0.746782   
                Innovative Funding Services, LLC       0.625874   
                Lending Arts                           0.611608   
                National Auto Loan Network             0.606013   
                RefiJet                                0.762996   
                Tresl                                  0.644007   
                Upfront                                0.523946   
                Yield Solutions Group                  0.685325   
                iLendingDIRECT                         0.647092   
current_partner N                                      0.616521   
                Y                                      0.726782   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.780545    71199.0   
Decision        Decline                                 0.716944    32611.0   
                Withdraw                                0.810673    38588.0   
fico_band       610-649                                 0.676105     6776.0   
                650-699                                 0.678492    16081.0   
                700-739                                 0.732695    15541.0   
                740-779                                 0.694296    15290.0   
                780-850                                 0.870052    17511.0   
Seller_Name     Auto Approve                            0.736465     9309.0   
                Autopay Direct, Inc.                    0.812489    37955.0   
                Caribou Financial Inc                   0.727925     3261.0   
                FiWize                                       NaN        1.0   
                Gravity Lending                         0.716673     2444.0   
                Innovative Funding Services, LLC        0.447552      146.0   
                Lending Arts                            0.690492      874.0   
                National Auto Loan Network              0.660398     4658.0   
                RefiJet                                 0.813551     3684.0   
                Tresl                                   0.733659     2620.0   
                Upfront                                 0.764368       93.0   
                Yield Solutions Group                   0.771248      929.0   
                iLendingDIRECT                          0.770512     5225.0   
current_partner N                                       0.698252     9321.0   
                Y                                       0.791229    61878.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             1746.0     0.024523  
Decision        Decline                              1222.0     0.037472  
                Withdraw                              524.0     0.013579  
fico_band       610-649

## 3. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2022 – Mar 2024


In [1011]:
# Most Recent 12 Month: April 2023- Mar 2024

start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'proxy_DQ120_m12'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m12


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.714054        0.803296   
Decision        Decline                          0.604751        0.734786   
                Withdraw                         0.742921        0.806456   
fico_band       610-649                          0.567643        0.730415   
                650-699                          0.550478        0.664405   
                700-739                          0.550230        0.717773   
                740-779                          0.595553        0.699242   
                780-850                          0.656478        0.896915   
Seller_Name     Auto Approve                     0.625065        0.775958   
                Autopay Direct, Inc.             0.771444        0.830077   
                Caribou Financial Inc            0.487678        0.617195   
                Gravity Lending                  0.577470        0.673452   
                Lending Arts                     0.767241        0.758621   
                National Auto Loan Network       0.517923        0.630149   
                RefiJet                          0.810473        0.819676   
                Tresl                                 NaN             NaN   
                iLendingDIRECT                   0.692339        0.810017   
current_partner N                                0.530877        0.640409   
                Y                                0.712845        0.805488   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      25361.0      276.0     0.010883  
Decision        Decline                        9709.0      188.0     0.019363  
                Withdraw                      15652.0       88.0     0.005622  
fico_band       610-649                         231.0       14.0     0.060606  
                650-699                        1761.0       48.0     0.027257  
                700-739                        4891.0       88.0     0.017992  
                740-779                        7525.0       84.0     0.011163  
                780-850                       10953.0       42.0     0.003835  
Seller_Name     Auto Approve                   4046.0       60.0     0.014829  
                Autopay Direct, Inc.          13922.0       95.0     0.006824  
                Caribou Financial Inc           895.0       25.0     0.027933  
                Gravity Lending                1277.0       12.0     0.009397  
                Lending Arts                     59.0        1.0     0.016949  
                National Auto Loan Network     1376.0       47.0     0.034157  
                RefiJet                        2612.0       17.0     0.006508  
                Tresl                            38.0        0.0     0.000000  
                iLendingDIRECT                 1136.0       19.0     0.016725  
current_partner N                              1473.0       48.0     0.032587  
                Y                             23888.0      228.0     0.009545

In [1012]:
# Full Time Window: Jan 2022 – Mar 2024

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'proxy_DQ120_m12'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m12


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.709789   
Decision        Decline                                0.625866   
                Withdraw                               0.740043   
fico_band       610-649                                0.538966   
                650-699                                0.550033   
                700-739                                0.547440   
                740-779                                0.567435   
                780-850                                0.651424   
Seller_Name     Auto Approve                           0.639061   
                Autopay Direct, Inc.                   0.757627   
                Caribou Financial Inc                  0.597972   
                FiWize                                      NaN   
                Gravity Lending                        0.743657   
                Innovative Funding Services, LLC       0.723958   
                Lending Arts                           0.603724   
                National Auto Loan Network             0.605892   
                RefiJet                                0.763289   
                Tresl                                  0.662954   
                Upfront                                0.487500   
                Yield Solutions Group                  0.678035   
                iLendingDIRECT                         0.631656   
current_partner N                                      0.619191   
                Y                                      0.720920   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.779736    71199.0   
Decision        Decline                                 0.717203    32611.0   
                Withdraw                                0.807141    38588.0   
fico_band       610-649                                 0.671934     6776.0   
                650-699                                 0.684264    16081.0   
                700-739                                 0.731610    15541.0   
                740-779                                 0.703832    15290.0   
                780-850                                 0.867615    17511.0   
Seller_Name     Auto Approve                            0.743502     9309.0   
                Autopay Direct, Inc.                    0.812622    37955.0   
                Caribou Financial Inc                   0.708501     3261.0   
                FiWize                                       NaN        1.0   
                Gravity Lending                         0.712680     2444.0   
                Innovative Funding Services, LLC        0.642361      146.0   
                Lending Arts                            0.662536      874.0   
                National Auto Loan Network              0.663323     4658.0   
                RefiJet                                 0.821419     3684.0   
                Tresl                                   0.732889     2620.0   
                Upfront                                 0.752273       93.0   
                Yield Solutions Group                   0.777796      929.0   
                iLendingDIRECT                          0.758329     5225.0   
current_partner N                                       0.700707     9321.0   
                Y                                       0.789767    61878.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             1575.0     0.022121  
Decision        Decline                              1104.0     0.033854  
                Withdraw                              471.0     0.012206  
fico_band       610-649

## 4. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023

In [1013]:
# Most Recent 12 Month: Oct 2022 - Sept 2023

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'proxy_DQ90_m18'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ90_m18


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.707798        0.776667   
Decision        Decline                          0.607983        0.706233   
                Withdraw                         0.723867        0.792222   
fico_band       610-649                          0.545728        0.668726   
                650-699                          0.527708        0.667179   
                700-739                          0.539062        0.688040   
                740-779                          0.564625        0.698131   
                780-850                          0.613087        0.857424   
Seller_Name     Auto Approve                     0.602953        0.716734   
                Autopay Direct, Inc.             0.758947        0.806067   
                Caribou Financial Inc            0.582857        0.723797   
                Gravity Lending                  0.635480        0.612595   
                Lending Arts                     0.727979        0.860241   
                National Auto Loan Network       0.530734        0.621520   
                RefiJet                          0.761287        0.785571   
                Tresl                            0.791139        0.924051   
                iLendingDIRECT                   0.621405        0.760106   
current_partner N                                0.569992        0.669658   
                Y                                0.710455        0.779722   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      25816.0      768.0     0.029749  
Decision        Decline                       10023.0      501.0     0.049985  
                Withdraw                      15793.0      267.0     0.016906  
fico_band       610-649                         383.0       38.0     0.099217  
                650-699                        3008.0      186.0     0.061835  
                700-739                        6517.0      296.0     0.045420  
                740-779                        7205.0      189.0     0.026232  
                780-850                        8703.0       59.0     0.006779  
Seller_Name     Auto Approve                   3463.0      139.0     0.040139  
                Autopay Direct, Inc.          14654.0      310.0     0.021155  
                Caribou Financial Inc          1065.0       54.0     0.050704  
                Gravity Lending                1022.0       32.0     0.031311  
                Lending Arts                    264.0       15.0     0.056818  
                National Auto Loan Network     1492.0      105.0     0.070375  
                RefiJet                        2148.0       55.0     0.025605  
                Tresl                            82.0        3.0     0.036585  
                iLendingDIRECT                 1626.0       55.0     0.033825  
current_partner N                              1838.0      123.0     0.066921  
                Y                             23978.0      645.0     0.026900

In [1014]:
# Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'proxy_DQ90_m18'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ90_m18


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.682191   
Decision        Decline                                0.598229   
                Withdraw                               0.709080   
fico_band       610-649                                0.541041   
                650-699                                0.546559   
                700-739                                0.537451   
                740-779                                0.549396   
                780-850                                0.640092   
Seller_Name     Auto Approve                           0.595976   
                Autopay Direct, Inc.                   0.726188   
                Caribou Financial Inc                  0.606387   
                FiWize                                      NaN   
                Gravity Lending                        0.665492   
                Innovative Funding Services, LLC       0.693662   
                Lending Arts                           0.642315   
                National Auto Loan Network             0.576823   
                RefiJet                                0.741055   
                Tresl                                  0.641673   
                Upfront                                0.531746   
                Yield Solutions Group                  0.657742   
                iLendingDIRECT                         0.647076   
current_partner N                                      0.602740   
                Y                                      0.694256   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.757363    59013.0   
Decision        Decline                                 0.693969    27740.0   
                Withdraw                                0.787174    31273.0   
fico_band       610-649                                 0.654103     6696.0   
                650-699                                 0.688916    15444.0   
                700-739                                 0.707757    13671.0   
                740-779                                 0.695036    11661.0   
                780-850                                 0.851456    11541.0   
Seller_Name     Auto Approve                            0.711992     6897.0   
                Autopay Direct, Inc.                    0.784487    31696.0   
                Caribou Financial Inc                   0.737792     2848.0   
                FiWize                                       NaN        1.0   
                Gravity Lending                         0.670520     1781.0   
                Innovative Funding Services, LLC        0.517606      146.0   
                Lending Arts                            0.727053      874.0   
                National Auto Loan Network              0.632296     4039.0   
                RefiJet                                 0.792736     2353.0   
                Tresl                                   0.748179     2618.0   
                Upfront                                 0.675926       93.0   
                Yield Solutions Group                   0.725029      929.0   
                iLendingDIRECT                          0.768440     4738.0   
current_partner N                                       0.686641     8700.0   
                Y                                       0.768472    50313.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             2782.0     0.047142  
Decision        Decline                              1927.0     0.069466  
                Withdraw                              855.0     0.027340  
fico_band       610-649

## 5. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023


In [1015]:
# Most Recent 12 Month: Oct 2022 - Sept 2023

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'proxy_DQ120_m18'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m18


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.706687        0.778012   
Decision        Decline                          0.605282        0.708666   
                Withdraw                         0.721177        0.791373   
fico_band       610-649                          0.593203        0.683636   
                650-699                          0.523293        0.667344   
                700-739                          0.541337        0.690276   
                740-779                          0.566359        0.699208   
                780-850                          0.604876        0.860847   
Seller_Name     Auto Approve                     0.598339        0.716386   
                Autopay Direct, Inc.             0.756284        0.809356   
                Caribou Financial Inc            0.587032        0.724201   
                Gravity Lending                  0.668716        0.626613   
                Lending Arts                     0.676428        0.803450   
                National Auto Loan Network       0.530550        0.620776   
                RefiJet                          0.767614        0.797358   
                Tresl                            0.791139        0.924051   
                iLendingDIRECT                   0.619809        0.753484   
current_partner N                                0.563118        0.657925   
                Y                                0.710504        0.782195   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      25816.0      717.0     0.027773  
Decision        Decline                       10023.0      468.0     0.046693  
                Withdraw                      15793.0      249.0     0.015766  
fico_band       610-649                         383.0       33.0     0.086162  
                650-699                        3008.0      173.0     0.057513  
                700-739                        6517.0      279.0     0.042811  
                740-779                        7205.0      178.0     0.024705  
                780-850                        8703.0       54.0     0.006205  
Seller_Name     Auto Approve                   3463.0      128.0     0.036962  
                Autopay Direct, Inc.          14654.0      293.0     0.019995  
                Caribou Financial Inc          1065.0       52.0     0.048826  
                Gravity Lending                1022.0       30.0     0.029354  
                Lending Arts                    264.0       11.0     0.041667  
                National Auto Loan Network     1492.0      100.0     0.067024  
                RefiJet                        2148.0       51.0     0.023743  
                Tresl                            82.0        3.0     0.036585  
                iLendingDIRECT                 1626.0       49.0     0.030135  
current_partner N                              1838.0      114.0     0.062024  
                Y                             23978.0      603.0     0.025148

In [1016]:
# Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'proxy_DQ120_m18'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m18


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.679957   
Decision        Decline                                0.595754   
                Withdraw                               0.705532   
fico_band       610-649                                0.540331   
                650-699                                0.548263   
                700-739                                0.544031   
                740-779                                0.548515   
                780-850                                0.635772   
Seller_Name     Auto Approve                           0.598634   
                Autopay Direct, Inc.                   0.724178   
                Caribou Financial Inc                  0.594376   
                FiWize                                      NaN   
                Gravity Lending                        0.675881   
                Innovative Funding Services, LLC       0.780886   
                Lending Arts                           0.631267   
                National Auto Loan Network             0.578830   
                RefiJet                                0.742492   
                Tresl                                  0.643963   
                Upfront                                0.521073   
                Yield Solutions Group                  0.661242   
                iLendingDIRECT                         0.635269   
current_partner N                                      0.603387   
                Y                                      0.691552   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.758520    59013.0   
Decision        Decline                                 0.697330    27740.0   
                Withdraw                                0.785328    31273.0   
fico_band       610-649                                 0.657857     6696.0   
                650-699                                 0.691667    15444.0   
                700-739                                 0.712869    13671.0   
                740-779                                 0.695745    11661.0   
                780-850                                 0.862696    11541.0   
Seller_Name     Auto Approve                            0.716957     6897.0   
                Autopay Direct, Inc.                    0.784880    31696.0   
                Caribou Financial Inc                   0.732918     2848.0   
                FiWize                                       NaN        1.0   
                Gravity Lending                         0.667445     1781.0   
                Innovative Funding Services, LLC        0.671329      146.0   
                Lending Arts                            0.700292      874.0   
                National Auto Loan Network              0.645634     4039.0   
                RefiJet                                 0.801038     2353.0   
                Tresl                                   0.756174     2618.0   
                Upfront                                 0.781609       93.0   
                Yield Solutions Group                   0.721181      929.0   
                iLendingDIRECT                          0.762213     4738.0   
current_partner N                                       0.694691     8700.0   
                Y                                       0.768355    50313.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             2489.0     0.042177  
Decision        Decline                              1727.0     0.062257  
                Withdraw                              762.0     0.024366  
fico_band       610-649

## 6. Most Recent 12 Month: April 2022 - Mar 2023; Full Time Window: Jan 2022 – Mar 2023


In [1017]:
# Most Recent 12 Month: April 2022 - Mar 2023

start_time = datetime.datetime(2022,4,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'proxy_DQ120_m24'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m24


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-04-01 to 2023-03-01         0.656548        0.736475   
Decision        Decline                          0.583570        0.684985   
                Withdraw                         0.675353        0.758925   
fico_band       610-649                          0.541101        0.653685   
                650-699                          0.538929        0.682960   
                700-739                          0.544538        0.694327   
                740-779                          0.546874        0.695957   
                780-850                          0.663685        0.835630   
Seller_Name     Auto Approve                     0.573915        0.701452   
                Autopay Direct, Inc.             0.699249        0.758614   
                Caribou Financial Inc            0.627331        0.702588   
                Gravity Lending                  0.681165        0.690782   
                Lending Arts                     0.646879        0.694243   
                National Auto Loan Network       0.591248        0.662315   
                RefiJet                          0.637366        0.801016   
                Tresl                            0.589721        0.723129   
                Upfront                          0.477196        0.668919   
                Yield Solutions Group            0.640300        0.731486   
                iLendingDIRECT                   0.629309        0.732588   
current_partner N                                0.597465        0.693334   
                Y                                0.667280        0.743784   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-04-01 to 2023-03-01      30172.0     2017.0     0.066850  
Decision        Decline                       14840.0     1370.0     0.092318  
                Withdraw                      15332.0      647.0     0.042199  
fico_band       610-649                        3782.0      441.0     0.116605  
                650-699                        9197.0      839.0     0.091225  
                700-739                        7361.0      473.0     0.064258  
                740-779                        5335.0      205.0     0.038425  
                780-850                        4497.0       59.0     0.013120  
Seller_Name     Auto Approve                   3872.0      252.0     0.065083  
                Autopay Direct, Inc.          15459.0      967.0     0.062553  
                Caribou Financial Inc          1390.0       89.0     0.064029  
                Gravity Lending                 688.0       44.0     0.063953  
                Lending Arts                    560.0       38.0     0.067857  
                National Auto Loan Network     2458.0      227.0     0.092352  
                RefiJet                         766.0       35.0     0.045692  
                Tresl                          1452.0       96.0     0.066116  
                Upfront                          82.0        8.0     0.097561  
                Yield Solutions Group           593.0       58.0     0.097808  
                iLendingDIRECT                 2852.0      203.0     0.071178  
current_partner N                              5145.0      427.0     0.082993  
                Y                             25027.0     1590.0     0.063531

In [1018]:
# Full Time Window: Jan 2022 – Mar 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'proxy_DQ120_m24'
fund_col = 'DATE_OPENED'
summary_df = performance_tables(df_unbook_w_targets, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

proxy_DQ120_m24


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-03-01               0.644857   
Decision        Decline                                0.570831   
                Withdraw                               0.670440   
fico_band       610-649                                0.538532   
                650-699                                0.540791   
                700-739                                0.538511   
                740-779                                0.544525   
                780-850                                0.664901   
Seller_Name     Auto Approve                           0.583086   
                Autopay Direct, Inc.                   0.673824   
                Caribou Financial Inc                  0.612825   
                FiWize                                      NaN   
                Gravity Lending                        0.655845   
                Innovative Funding Services, LLC       0.592908   
                Lending Arts                           0.632339   
                National Auto Loan Network             0.587353   
                RefiJet                                0.637366   
                Tresl                                  0.636082   
                Upfront                                0.504412   
                Yield Solutions Group                  0.624905   
                iLendingDIRECT                         0.617427   
current_partner N                                      0.604712   
                Y                                      0.653391   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-03-01                0.733439    42575.0   
Decision        Decline                                 0.682801    21630.0   
                Withdraw                                0.760181    20945.0   
fico_band       610-649                                 0.653866     6501.0   
                650-699                                 0.684758    13914.0   
                700-739                                 0.707373     9787.0   
                740-779                                 0.699657     6881.0   
                780-850                                 0.827388     5492.0   
Seller_Name     Auto Approve                            0.702172     4869.0   
                Autopay Direct, Inc.                    0.749580    22048.0   
                Caribou Financial Inc                   0.727218     2238.0   
                FiWize                                       NaN        1.0   
                Gravity Lending                         0.677291     1026.0   
                Innovative Funding Services, LLC        0.690780      146.0   
                Lending Arts                            0.666813      781.0   
                National Auto Loan Network              0.656390     3173.0   
                RefiJet                                 0.801016      766.0   
                Tresl                                   0.746112     2578.0   
                Upfront                                 0.679412       93.0   
                Yield Solutions Group                   0.717695      929.0   
                iLendingDIRECT                          0.744047     3927.0   
current_partner N                                       0.697324     7701.0   
                Y                                       0.740908    34874.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-03-01             2840.0     0.066706  
Decision        Decline                              1954.0     0.090337  
                Withdraw                              886.0     0.042301  
fico_band       610-649

# Overall Performance

In [1019]:
df_unbook_renamed = df_unbook_w_targets.drop(columns = ['Funded_Date',
    'mob_6_60', 'mob_12_90', 'mob_18_90', 'mob_24_120', 'mob_12_120', 'mob_18_120'])

In [1020]:
df_unbook_renamed = df_unbook_renamed.rename(columns = {'proxy_DQ60_m6': 'mob_6_60',
                  'proxy_DQ90_m12': 'mob_12_90',
                  'proxy_DQ120_m12': 'mob_12_120',
                  'proxy_DQ90_m18': 'mob_18_90',
                  'proxy_DQ120_m18': 'mob_18_120',
                  'proxy_DQ120_m24': 'mob_24_120',
                  'DATE_OPENED': 'Funded_Date'})

In [1021]:
common_cols = ['ZEST_KEY', 'appId', 'appDate', 'flgFunded', 'flgApproved',
               'mob_6_60', 'mob_12_90', 'mob_12_120','mob_18_90','mob_18_120','mob_24_120',
               'Decision', 'fico_band', 'Seller_Name','current_partner', 'Funded_Date',
               'inverted_benchmark_scores','final_model_predictions', 'benchmark_scores',
              ]

In [1022]:
df['flgFunded'].value_counts()

flgFunded
0    407464
1    109650
Name: count, dtype: int64

In [1023]:
df_book.shape, df_unbook_renamed.shape

((109650, 189), (407464, 199))

In [1024]:
df_book_udpated = df_book.loc[:, ~df_book.columns.duplicated()]
df_unbook_renamed = df_unbook_renamed.loc[:, ~df_unbook_renamed.columns.duplicated()]
df_book_udpated.shape, df_unbook_renamed.shape

((109650, 189), (407464, 199))

In [1025]:
df_combined = pd.concat([
    df_book_udpated.reset_index()[common_cols], 
    df_unbook_renamed[common_cols]
], ignore_index=True)

In [1026]:
df_combined['Funded_Date'] = pd.to_datetime(df_combined['Funded_Date'])
target_cols = ['mob_6_60', 'mob_12_90', 'mob_18_90', 'mob_24_120', 'mob_12_120', 'mob_18_120']
for i in target_cols: 
    df_combined[i] = pd.to_numeric(df_combined[i], errors='coerce')


## 1. Most Recent 12 Month: Oct 2023 - Sept 2024

In [1027]:
# Most Recent 12 Month: Oct 2023 - Sept 2024

start_time = datetime.datetime(2023,10,1)
end_time = datetime.datetime(2024,9,30)

target_column = 'mob_6_60'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_6_60


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-10-01 to 2024-09-30         0.699026        0.772404   
Decision        Decline                          0.596889        0.712459   
                Funded                           0.689254        0.748468   
                Withdraw                         0.734585        0.781102   
fico_band       610-649                          0.459936        0.477564   
                650-699                          0.424051        0.714375   
                700-739                          0.580704        0.643028   
                740-779                          0.614376        0.706895   
                780-850                          0.684572        0.771601   
Seller_Name     Auto Approve                     0.666359        0.691006   
                Autopay Direct, Inc.             0.706820        0.813313   
                Caribou Financial Inc            0.567161        0.400180   
                Gravity Lending                  0.857860        0.820105   
                National Auto Loan Network       0.661918        0.685436   
                RefiJet                          0.598593        0.894558   
                Tresl                                 NaN             NaN   
                iLendingDIRECT                   0.516894        0.597141   
current_partner N                                0.662376        0.685863   
                Y                                0.692541        0.769763   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-10-01 to 2024-09-30      35497.0      106.0     0.002986  
Decision        Decline                       10132.0       58.0     0.005724  
                Funded                        11591.0       30.0     0.002588  
                Withdraw                      13774.0       18.0     0.001307  
fico_band       610-649                         107.0        3.0     0.028037  
                650-699                         943.0       12.0     0.012725  
                700-739                        2886.0       15.0     0.005198  
                740-779                       10526.0       39.0     0.003705  
                780-850                       21035.0       37.0     0.001759  
Seller_Name     Auto Approve                   6812.0       30.0     0.004404  
                Autopay Direct, Inc.          20474.0       52.0     0.002540  
                Caribou Financial Inc          1597.0        7.0     0.004383  
                Gravity Lending                1623.0        4.0     0.002465  
                National Auto Loan Network      743.0        6.0     0.008075  
                RefiJet                        2706.0        5.0     0.001848  
                Tresl                             1.0        0.0     0.000000  
                iLendingDIRECT                 1541.0        2.0     0.001298  
current_partner N                               744.0        6.0     0.008065  
                Y                             34753.0      100.0     0.002877

## 2. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2021 – Mar 2024


In [1028]:
# Full Time Window: Jan 2021 – Mar 2024

# Most Recent 12 Month: April 2023 - Mar 2024

start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_90


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.706601        0.783610   
Decision        Decline                          0.599670        0.731769   
                Funded                           0.707803        0.759528   
                Withdraw                         0.750507        0.809893   
fico_band       610-649                          0.568480        0.731651   
                650-699                          0.549994        0.662324   
                700-739                          0.563737        0.695525   
                740-779                          0.572117        0.688033   
                780-850                          0.656685        0.845930   
Seller_Name     Auto Approve                     0.639002        0.739195   
                Autopay Direct, Inc.             0.747599        0.818398   
                Caribou Financial Inc            0.578753        0.686805   
                Gravity Lending                  0.633359        0.716021   
                Lending Arts                     0.640244        0.560976   
                National Auto Loan Network       0.532250        0.655709   
                RefiJet                          0.750574        0.745591   
                Tresl                            0.576087        0.956522   
                iLendingDIRECT                   0.708823        0.783745   
current_partner N                                0.538184        0.657271   
                Y                                0.705382        0.782788   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      42560.0      550.0     0.012923  
Decision        Decline                        9709.0      195.0     0.020084  
                Funded                        17199.0      262.0     0.015233  
                Withdraw                      15652.0       93.0     0.005942  
fico_band       610-649                         232.0       14.0     0.060345  
                650-699                        1780.0       50.0     0.028090  
                700-739                        7727.0      195.0     0.025236  
                740-779                       14704.0      212.0     0.014418  
                780-850                       18117.0       79.0     0.004361  
Seller_Name     Auto Approve                   8329.0      136.0     0.016328  
                Autopay Direct, Inc.          20943.0      176.0     0.008404  
                Caribou Financial Inc          3028.0       62.0     0.020476  
                Gravity Lending                2079.0       21.0     0.010101  
                Lending Arts                     85.0        3.0     0.035294  
                National Auto Loan Network     1684.0       63.0     0.037411  
                RefiJet                        3785.0       36.0     0.009511  
                Tresl                            47.0        1.0     0.021277  
                iLendingDIRECT                 2580.0       52.0     0.020155  
current_partner N                              1816.0       67.0     0.036894  
                Y                             40744.0      483.0     0.011855

In [1029]:
# Full Time Window: Jan 2021 – Mar 2024

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_90


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.688089   
Decision        Decline                                0.630217   
                Funded                                 0.654070   
                Withdraw                               0.747112   
fico_band       610-649                                0.541395   
                650-699                                0.543085   
                700-739                                0.539759   
                740-779                                0.546461   
                780-850                                0.658613   
Seller_Name     Auto Approve                           0.632331   
                Autopay Direct, Inc.                   0.728535   
                Caribou Financial Inc                  0.631704   
                FiWize                                      NaN   
                Gravity Lending                        0.737083   
                Innovative Funding Services, LLC       0.682487   
                Lending Arts                           0.595023   
                National Auto Loan Network             0.600136   
                RefiJet                                0.716380   
                Tresl                                  0.642881   
                Upfront                                0.623303   
                Yield Solutions Group                  0.659021   
                iLendingDIRECT                         0.657482   
current_partner N                                      0.619818   
                Y                                      0.694966   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.757818   136952.0   
Decision        Decline                                 0.716944    32611.0   
                Funded                                  0.730658    65753.0   
                Withdraw                                0.810673    38588.0   
fico_band       610-649                                 0.675029     6832.0   
                650-699                                 0.675526    30993.0   
                700-739                                 0.696753    35752.0   
                740-779                                 0.701506    33435.0   
                780-850                                 0.815826    29940.0   
Seller_Name     Auto Approve                            0.701691    20470.0   
                Autopay Direct, Inc.                    0.789928    62596.0   
                Caribou Financial Inc                   0.735424    14170.0   
                FiWize                                       NaN        6.0   
                Gravity Lending                         0.713414     4947.0   
                Innovative Funding Services, LLC        0.491979      191.0   
                Lending Arts                            0.677988     1995.0   
                National Auto Loan Network              0.663275     6972.0   
                RefiJet                                 0.758257     5647.0   
                Tresl                                   0.755955     4760.0   
                Upfront                                 0.756561      231.0   
                Yield Solutions Group                   0.735709     2072.0   
                iLendingDIRECT                          0.758727    12895.0   
current_partner N                                       0.705467    16227.0   
                Y                                       0.763088   120725.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             3302.0     0.024111  
Decision        Decline    

## 3. Most Recent 12 Month: April 2023 - Mar 2024; Full Time Window: Jan 2022 – Mar 2024


In [1030]:
# Most Recent 12 Month: April 2023 - Mar 2024


start_time = datetime.datetime(2023,4,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2023-04-01 to 2024-03-01         0.705535        0.781677   
Decision        Decline                          0.604751        0.734786   
                Funded                           0.704373        0.754549   
                Withdraw                         0.742921        0.806456   
fico_band       610-649                          0.568480        0.731651   
                650-699                          0.546782        0.666301   
                700-739                          0.568121        0.687091   
                740-779                          0.570485        0.686817   
                780-850                          0.657974        0.844470   
Seller_Name     Auto Approve                     0.640215        0.736646   
                Autopay Direct, Inc.             0.749692        0.818399   
                Caribou Financial Inc            0.571993        0.686854   
                Gravity Lending                  0.624660        0.701870   
                Lending Arts                     0.640244        0.560976   
                National Auto Loan Network       0.534581        0.646567   
                RefiJet                          0.752372        0.753336   
                Tresl                            0.576087        0.956522   
                iLendingDIRECT                   0.703312        0.780206   
current_partner N                                0.539975        0.648125   
                Y                                0.705139        0.782020   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2023-04-01 to 2024-03-01      42560.0      526.0     0.012359  
Decision        Decline                        9709.0      188.0     0.019363  
                Funded                        17199.0      250.0     0.014536  
                Withdraw                      15652.0       88.0     0.005622  
fico_band       610-649                         232.0       14.0     0.060345  
                650-699                        1780.0       49.0     0.027528  
                700-739                        7727.0      185.0     0.023942  
                740-779                       14704.0      200.0     0.013602  
                780-850                       18117.0       78.0     0.004305  
Seller_Name     Auto Approve                   8329.0      133.0     0.015968  
                Autopay Direct, Inc.          20943.0      168.0     0.008022  
                Caribou Financial Inc          3028.0       60.0     0.019815  
                Gravity Lending                2079.0       20.0     0.009620  
                Lending Arts                     85.0        3.0     0.035294  
                National Auto Loan Network     1684.0       57.0     0.033848  
                RefiJet                        3785.0       34.0     0.008983  
                Tresl                            47.0        1.0     0.021277  
                iLendingDIRECT                 2580.0       50.0     0.019380  
current_partner N                              1816.0       61.0     0.033590  
                Y                             40744.0      465.0     0.011413

In [1031]:
# Full Time Window: Jan 2022 – Mar 2024


start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2024,3,1)

target_column = 'mob_12_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_12_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2024-03-01               0.681471   
Decision        Decline                                0.625866   
                Funded                                 0.638121   
                Withdraw                               0.740043   
fico_band       610-649                                0.538761   
                650-699                                0.554621   
                700-739                                0.534129   
                740-779                                0.539450   
                780-850                                0.660152   
Seller_Name     Auto Approve                           0.629031   
                Autopay Direct, Inc.                   0.723879   
                Caribou Financial Inc                  0.605446   
                FiWize                                      NaN   
                Gravity Lending                        0.734361   
                Innovative Funding Services, LLC       0.773810   
                Lending Arts                           0.596140   
                National Auto Loan Network             0.600777   
                RefiJet                                0.721092   
                Tresl                                  0.664185   
                Upfront                                0.579719   
                Yield Solutions Group                  0.655384   
                iLendingDIRECT                         0.640440   
current_partner N                                      0.622589   
                Y                                      0.686620   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2024-03-01                0.755178   136952.0   
Decision        Decline                                 0.717203    32611.0   
                Funded                                  0.722117    65753.0   
                Withdraw                                0.807141    38588.0   
fico_band       610-649                                 0.670720     6832.0   
                650-699                                 0.678950    30993.0   
                700-739                                 0.695821    35752.0   
                740-779                                 0.702198    33435.0   
                780-850                                 0.816947    29940.0   
Seller_Name     Auto Approve                            0.704890    20470.0   
                Autopay Direct, Inc.                    0.790731    62596.0   
                Caribou Financial Inc                   0.722303    14170.0   
                FiWize                                       NaN        6.0   
                Gravity Lending                         0.705865     4947.0   
                Innovative Funding Services, LLC        0.701058      191.0   
                Lending Arts                            0.657377     1995.0   
                National Auto Loan Network              0.662404     6972.0   
                RefiJet                                 0.760917     5647.0   
                Tresl                                   0.742110     4760.0   
                Upfront                                 0.803571      231.0   
                Yield Solutions Group                   0.739328     2072.0   
                iLendingDIRECT                          0.747757    12895.0   
current_partner N                                       0.703198    16227.0   
                Y                                       0.760201   120725.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2024-03-01             2848.0     0.020796  
Decision        Decline    

## 4. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023


In [1032]:
# Most Recent 12 Month: Oct 2022 - Sept 2023

target_column = 'mob_18_90'

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_90


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.673557        0.741542   
Decision        Decline                          0.607983        0.706233   
                Funded                           0.638698        0.700792   
                Withdraw                         0.723867        0.792222   
fico_band       610-649                          0.546281        0.669531   
                650-699                          0.529725        0.667226   
                700-739                          0.541184        0.664636   
                740-779                          0.543669        0.678939   
                780-850                          0.640358        0.799928   
Seller_Name     Auto Approve                     0.608147        0.685511   
                Autopay Direct, Inc.             0.713918        0.777776   
                Caribou Financial Inc            0.604260        0.697916   
                Gravity Lending                  0.633781        0.634911   
                Lending Arts                     0.737936        0.814190   
                National Auto Loan Network       0.530487        0.615703   
                RefiJet                          0.713809        0.741269   
                Tresl                            0.671000        0.924000   
                iLendingDIRECT                   0.639226        0.736290   
current_partner N                                0.571285        0.663261   
                Y                                0.675408        0.743344   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      46965.0     1628.0     0.034664  
Decision        Decline                       10023.0      501.0     0.049985  
                Funded                        21149.0      860.0     0.040664  
                Withdraw                      15793.0      267.0     0.016906  
fico_band       610-649                         384.0       38.0     0.098958  
                650-699                        3044.0      191.0     0.062746  
                700-739                       14697.0      789.0     0.053684  
                740-779                       15121.0      478.0     0.031612  
                780-850                       13719.0      132.0     0.009622  
Seller_Name     Auto Approve                   8716.0      384.0     0.044057  
                Autopay Direct, Inc.          22285.0      560.0     0.025129  
                Caribou Financial Inc          4001.0      160.0     0.039990  
                Gravity Lending                1834.0       57.0     0.031080  
                Lending Arts                    470.0       21.0     0.044681  
                National Auto Loan Network     2010.0      143.0     0.071144  
                RefiJet                        3439.0      108.0     0.031404  
                Tresl                           105.0        5.0     0.047619  
                iLendingDIRECT                 4105.0      190.0     0.046285  
current_partner N                              2585.0      169.0     0.065377  
                Y                             44380.0     1459.0     0.032875

In [1033]:
#  Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'mob_18_90'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_90


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.658262   
Decision        Decline                                0.598229   
                Funded                                 0.632705   
                Withdraw                               0.709080   
fico_band       610-649                                0.540649   
                650-699                                0.539928   
                700-739                                0.534656   
                740-779                                0.529692   
                780-850                                0.646123   
Seller_Name     Auto Approve                           0.600835   
                Autopay Direct, Inc.                   0.691385   
                Caribou Financial Inc                  0.628669   
                FiWize                                      NaN   
                Gravity Lending                        0.656613   
                Innovative Funding Services, LLC       0.731183   
                Lending Arts                           0.610638   
                National Auto Loan Network             0.575436   
                RefiJet                                0.700271   
                Tresl                                  0.639096   
                Upfront                                0.613808   
                Yield Solutions Group                  0.642235   
                iLendingDIRECT                         0.636930   
current_partner N                                      0.608929   
                Y                                      0.664139   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.737991   117419.0   
Decision        Decline                                 0.693969    27740.0   
                Funded                                  0.718119    58406.0   
                Withdraw                                0.787174    31273.0   
fico_band       610-649                                 0.652122     6752.0   
                650-699                                 0.679362    30349.0   
                700-739                                 0.689855    33814.0   
                740-779                                 0.697275    26535.0   
                780-850                                 0.789644    19969.0   
Seller_Name     Auto Approve                            0.688093    16101.0   
                Autopay Direct, Inc.                    0.762574    53373.0   
                Caribou Financial Inc                   0.733050    12912.0   
                FiWize                                       NaN        6.0   
                Gravity Lending                         0.673577     3898.0   
                Innovative Funding Services, LLC        0.550538      191.0   
                Lending Arts                            0.717174     1995.0   
                National Auto Loan Network              0.640793     6250.0   
                RefiJet                                 0.733557     3801.0   
                Tresl                                   0.759067     4758.0   
                Upfront                                 0.747674      231.0   
                Yield Solutions Group                   0.710361     2072.0   
                iLendingDIRECT                          0.743266    11831.0   
current_partner N                                       0.699695    15503.0   
                Y                                       0.742552   101916.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             5568.0     0.047420  
Decision        Decline    

## 5. Most Recent 12 Month: Oct 2022 - Sept 2023; Full Time Window: Jan 2022 – Sept 2023



In [1034]:
# Most Recent 12 Month: Oct 2022 - Sept 2023
target_column = 'mob_18_120'

start_time = datetime.datetime(2022,10,1)
end_time = datetime.datetime(2023,9,1)

fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-10-01 to 2023-09-01         0.672664        0.741250   
Decision        Decline                          0.605282        0.708666   
                Funded                           0.638188        0.698832   
                Withdraw                         0.721177        0.791373   
fico_band       610-649                          0.593844        0.684451   
                650-699                          0.525531        0.667373   
                700-739                          0.541543        0.662778   
                740-779                          0.546853        0.679171   
                780-850                          0.641160        0.801268   
Seller_Name     Auto Approve                     0.604059        0.685858   
                Autopay Direct, Inc.             0.713745        0.780371   
                Caribou Financial Inc            0.605342        0.697603   
                Gravity Lending                  0.649844        0.634993   
                Lending Arts                     0.721503        0.768722   
                National Auto Loan Network       0.534627        0.619129   
                RefiJet                          0.716931        0.741647   
                Tresl                            0.671000        0.924000   
                iLendingDIRECT                   0.634706        0.727998   
current_partner N                                0.571349        0.657897   
                Y                                0.674599        0.743497   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-10-01 to 2023-09-01      46965.0     1528.0     0.032535  
Decision        Decline                       10023.0      468.0     0.046693  
                Funded                        21149.0      811.0     0.038347  
                Withdraw                      15793.0      249.0     0.015766  
fico_band       610-649                         384.0       33.0     0.085938  
                650-699                        3044.0      178.0     0.058476  
                700-739                       14697.0      742.0     0.050486  
                740-779                       15121.0      452.0     0.029892  
                780-850                       13719.0      123.0     0.008966  
Seller_Name     Auto Approve                   8716.0      360.0     0.041303  
                Autopay Direct, Inc.          22285.0      529.0     0.023738  
                Caribou Financial Inc          4001.0      156.0     0.038990  
                Gravity Lending                1834.0       54.0     0.029444  
                Lending Arts                    470.0       16.0     0.034043  
                National Auto Loan Network     2010.0      135.0     0.067164  
                RefiJet                        3439.0       99.0     0.028787  
                Tresl                           105.0        5.0     0.047619  
                iLendingDIRECT                 4105.0      174.0     0.042387  
current_partner N                              2585.0      156.0     0.060348  
                Y                             44380.0     1372.0     0.030915

In [1035]:
#  Full Time Window: Jan 2022 – Sept 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,9,1)

target_column = 'mob_18_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_18_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-09-01               0.652224   
Decision        Decline                                0.595754   
                Funded                                 0.619596   
                Withdraw                               0.705532   
fico_band       610-649                                0.539832   
                650-699                                0.541459   
                700-739                                0.534217   
                740-779                                0.532489   
                780-850                                0.647610   
Seller_Name     Auto Approve                           0.595402   
                Autopay Direct, Inc.                   0.686902   
                Caribou Financial Inc                  0.615041   
                FiWize                                      NaN   
                Gravity Lending                        0.656060   
                Innovative Funding Services, LLC       0.793449   
                Lending Arts                           0.602820   
                National Auto Loan Network             0.575983   
                RefiJet                                0.700698   
                Tresl                                  0.641032   
                Upfront                                0.574011   
                Yield Solutions Group                  0.654046   
                iLendingDIRECT                         0.625745   
current_partner N                                      0.609542   
                Y                                      0.657177   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-09-01                0.736383   117419.0   
Decision        Decline                                 0.697330    27740.0   
                Funded                                  0.712310    58406.0   
                Withdraw                                0.785328    31273.0   
fico_band       610-649                                 0.655579     6752.0   
                650-699                                 0.683431    30349.0   
                700-739                                 0.690819    33814.0   
                740-779                                 0.694543    26535.0   
                780-850                                 0.795627    19969.0   
Seller_Name     Auto Approve                            0.690252    16101.0   
                Autopay Direct, Inc.                    0.761948    53373.0   
                Caribou Financial Inc                   0.726860    12912.0   
                FiWize                                       NaN        6.0   
                Gravity Lending                         0.661791     3898.0   
                Innovative Funding Services, LLC        0.665775      191.0   
                Lending Arts                            0.698956     1995.0   
                National Auto Loan Network              0.650855     6250.0   
                RefiJet                                 0.732467     3801.0   
                Tresl                                   0.757404     4758.0   
                Upfront                                 0.801370      231.0   
                Yield Solutions Group                   0.704507     2072.0   
                iLendingDIRECT                          0.739626    11831.0   
current_partner N                                       0.701898    15503.0   
                Y                                       0.740377   101916.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-09-01             4901.0     0.041739  
Decision        Decline    

## 6. Most Recent 12 Month: April 2022 - Mar 2023; Full Time Window: Jan 2022 – Mar 2023

In [1036]:
# Most Recent 12 Month: April 2022 - Mar 2023

start_time = datetime.datetime(2022,4,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'mob_24_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_24_120


benchmark_auc  zest_model_auc  \
grouping        group                                                       
Overall         2022-04-01 to 2023-03-01         0.629902        0.714975   
Decision        Decline                          0.583570        0.684985   
                Funded                           0.605697        0.695282   
                Withdraw                         0.675353        0.758925   
fico_band       610-649                          0.540530        0.652542   
                650-699                          0.538673        0.671369   
                700-739                          0.541451        0.681097   
                740-779                          0.530671        0.692606   
                780-850                          0.620100        0.770042   
Seller_Name     Auto Approve                     0.586866        0.685643   
                Autopay Direct, Inc.             0.659005        0.732633   
                Caribou Financial Inc            0.619384        0.706487   
                Gravity Lending                  0.619909        0.644552   
                Lending Arts                     0.606012        0.710215   
                National Auto Loan Network       0.574104        0.660050   
                RefiJet                          0.607705        0.717133   
                Tresl                            0.590578        0.715924   
                Upfront                          0.620059        0.767956   
                Yield Solutions Group            0.621567        0.701112   
                iLendingDIRECT                   0.616573        0.713086   
current_partner N                                0.593787        0.694906   
                Y                                0.634541        0.716950   

                                            app_count  bad_count  target_rate  
grouping        group                                                          
Overall         2022-04-01 to 2023-03-01      59508.0     4146.0     0.069671  
Decision        Decline                       14840.0     1370.0     0.092318  
                Funded                        29336.0     2129.0     0.072573  
                Withdraw                      15332.0      647.0     0.042199  
fico_band       610-649                        3810.0      444.0     0.116535  
                650-699                       17208.0     1670.0     0.097048  
                700-739                       18808.0     1321.0     0.070236  
                740-779                       12314.0      565.0     0.045883  
                780-850                        7368.0      146.0     0.019815  
Seller_Name     Auto Approve                   8568.0      587.0     0.068511  
                Autopay Direct, Inc.          26411.0     1738.0     0.065806  
                Caribou Financial Inc          6079.0      389.0     0.063991  
                Gravity Lending                1549.0       95.0     0.061330  
                Lending Arts                   1181.0       67.0     0.056732  
                National Auto Loan Network     3736.0      382.0     0.102248  
                RefiJet                        1394.0       80.0     0.057389  
                Tresl                          2379.0      156.0     0.065574  
                Upfront                         194.0       13.0     0.067010  
                Yield Solutions Group          1212.0      113.0     0.093234  
                iLendingDIRECT                 6805.0      526.0     0.077296  
current_partner N                              8702.0      731.0     0.084004  
                Y                             50806.0     3415.0     0.067216

In [1037]:
# Full Time Window: Jan 2022 – Mar 2023

start_time = datetime.datetime(2022,1,1)
end_time = datetime.datetime(2023,3,1)

target_column = 'mob_24_120'
fund_col = 'Funded_Date'
summary_df = performance_tables(df_combined, fund_col, start_time, end_time, target_column, 
                                decision_seg = True, fico_seg = True, seller_seg = True, 
                                partner_seg = True)
summary_df

mob_24_120


benchmark_auc  \
grouping        group                                             
Overall         2022-01-01 to 2023-03-01               0.628938   
Decision        Decline                                0.570831   
                Funded                                 0.614911   
                Withdraw                               0.670440   
fico_band       610-649                                0.538525   
                650-699                                0.539655   
                700-739                                0.538637   
                740-779                                0.525868   
                780-850                                0.628340   
Seller_Name     Auto Approve                           0.591739   
                Autopay Direct, Inc.                   0.645224   
                Caribou Financial Inc                  0.616751   
                FiWize                                      NaN   
                Gravity Lending                        0.646626   
                Innovative Funding Services, LLC       0.651802   
                Lending Arts                           0.599864   
                National Auto Loan Network             0.575151   
                RefiJet                                0.607705   
                Tresl                                  0.629446   
                Upfront                                0.598692   
                Yield Solutions Group                  0.634463   
                iLendingDIRECT                         0.616998   
current_partner N                                      0.608173   
                Y                                      0.632384   

                                                  zest_model_auc  app_count  \
grouping        group                                                         
Overall         2022-01-01 to 2023-03-01                0.718981    88392.0   
Decision        Decline                                 0.682801    21630.0   
                Funded                                  0.706846    45817.0   
                Withdraw                                0.760181    20945.0   
fico_band       610-649                                 0.652465     6556.0   
                650-699                                 0.677495    28800.0   
                700-739                                 0.686613    25972.0   
                740-779                                 0.703078    16825.0   
                780-850                                 0.762413    10239.0   
Seller_Name     Auto Approve                            0.689493    11037.0   
                Autopay Direct, Inc.                    0.730916    38624.0   
                Caribou Financial Inc                   0.713840    10676.0   
                FiWize                                       NaN        6.0   
                Gravity Lending                         0.679545     2611.0   
                Innovative Funding Services, LLC        0.702703      191.0   
                Lending Arts                            0.694521     1842.0   
                National Auto Loan Network              0.653727     5126.0   
                RefiJet                                 0.717133     1394.0   
                Tresl                                   0.742276     4706.0   
                Upfront                                 0.753198      231.0   
                Yield Solutions Group                   0.708737     2072.0   
                iLendingDIRECT                          0.722377     9876.0   
current_partner N                                       0.703772    14174.0   
                Y                                       0.721313    74218.0   

                                                  bad_count  target_rate  
grouping        group                                                     
Overall         2022-01-01 to 2023-03-01             5913.0     0.066895  
Decision        Decline    